<a href="https://colab.research.google.com/github/jonahsjlee/programming/blob/main/kkn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Q0

What is the difference between regression and classification?
- Regression is used for predicting quantities. Classification is for assigning items to categories.


What is a confusion table? What does it help us understand about a model's performance?
- A performance measurement tool used for classification models. It shows how well the model differentiates between classes and helps identify potential issues like bias toward one class or the need to balance false positives and false negatives in certain applications.

What does the SSE quantify about a particular model? - It quantifies the overall error or deviation of a model’s predicted values from the actual observed values.

  - What are overfitting and underfitting? - Overfitting occurs when a model is too complex and fits the training data too closely, capturing noise and random fluctuations rather than the true underlying patterns. Underfitting occurs when a model is too simple to capture the underlying patterns in the data, leading to poor performance on both the training data and unseen data.

Why does splitting the data into training and testing sets, and choosing
 by evaluating accuracy or SSE on the test set, improve model performance?
 -  Splitting the data into training and testing sets and evaluating model performance on the test set helps improve a model's ability to generalize to unseen data

With classification, we can report a class label as a prediction or a probability distribution over class labels. Please explain the strengths and weaknesses of each approach.
- Class label predictions can be easily interpreted, but can be unclear how certain the prediction is if probabilities are not reported.  

Q1

1. Load the ./data/USA_cars_datasets.csv. Keep the following variables and drop the rest: price, year, mileage. Are there any NA's to handle? Look at the head and dimensions of the data.

In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv('./data/USA_cars_datasets.csv')
df = df.loc[:,['price','year','mileage'] ]
print(df.shape)
print(df.describe())
df.head()

FileNotFoundError: [Errno 2] No such file or directory: './data/USA_cars_datasets.csv'

2. Maxmin normalize year and mileage.

In [ ]:
def maxmin(x):
    u = (x-min(x))/(max(x)-min(x))
    return u

df['year'] = maxmin(df['year'])
df['mileage'] = maxmin(df['mileage'])

3. Split the sample into ~80% for training and ~20% for evaluation.

In [ ]:
from sklearn.model_selection import train_test_split
y = df['price']
X = df.drop('price',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2,random_state=100)

4. Use the
NN algorithm and the training data to predict price using year and mileage for the test set for
k = 3, 10, 25, 50, 100, 300. For each value of
, compute the mean squared error and print a scatterplot showing the test value plotted against the predicted value. What patterns do you notice as you increase k?

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt

for k in [3,10,25,50,100,300]:
    model = KNeighborsRegressor(n_neighbors=k).fit(X_train,y_train)
    y_hat = model.predict(X_test)
    SSE = np.sum( (y_test-y_hat)**2 )
    #
    plot, axes = plt.subplots()
    plt.scatter(y_test,y_hat)
    plt.title('k: '+str(k)+', SSE: '+str(SSE))
    axes.set_ylim(-1000, 62000)
    axes.set_xlim(-1000, 62000)
    plt.show()

5. Determine the optimal k
 for these data.

In [ ]:
k_bar = 200
k_grid = np.arange(1,k_bar) # The range of k's to consider
SSE = np.zeros(k_bar)

for k in range(k_bar):
    fitted_model = KNeighborsRegressor(n_neighbors=k+1).fit(X_train,y_train)
    y_hat = fitted_model.predict(X_test) # Predict values for test set
    SSE[k] = np.sum( (y_test-y_hat)**2 ) # Save the computed SSE

SSE_min = np.min(SSE) # Lowest recorded SSE
min_index = np.where(SSE==SSE_min) # Find the indices of y that equal the minimum
k_star = k_grid[min_index] # Find the optimal value of k
print(k_star)

plt.plot(np.arange(0,k_bar),SSE) # Plot SSE by k
plt.xlabel("k")
plt.title("optimal k:"+str(k_star))
plt.ylabel('SSE')
plt.show()

6. Describe what happened in the plots of predicted versus actual prices as
k varied, taking your answer into part 6 into account. (Hint: Use the words "underfitting" and "overfitting".)

- The optimal $k$ is around 77, so for 300, the model is probably overfitting, and for 3, 10, and 25, the model is underfitting. For $k$ equal to 50 and 100, the answer is pretty close. What do we notice about $k=300$? There is a bunch of horizontal bunching, where common answers start to exert a lot of influence on the predictions and organize them into horizontal groups. Since the data are fairly evenly distributed, this is an unnatural artifact of over-fitting. On the other hand, for very small $k$, like 3, the predictions are high variance and low precision.

- The other thing to notice is that as $k$ increases, the range of predictions shrinks: The high, outlier values become less influential, and predictions shrink towards average values.